In [7]:
import numpy as np
import matplotlib.pyplot as plt
import datajoint as dj
import pandas as pd
import pandas as pd
from pathlib import Path

from io_utils import *
%load_ext autoreload
%autoreload 2
dj.blob.use_32bit_dims = True # necessary for pd.blob read

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
#### MULTI SESSION
animal_id = 'R502'
bdata = dj.create_virtual_module("new_acquisition", "bdatatest")
subject_session_key = {"ratname": animal_id}


protocol_blobs = (bdata.Sessions & subject_session_key).fetch(
    "protocol_data", as_dict=True
)

sess_ids, dates, trials = (bdata.Sessions & subject_session_key).fetch(
    "sessid", "sessiondate", "n_done_trials"
)

protocol_blobs, sess_ids, dates, trials = drop_empty_sessions(
    protocol_blobs, sess_ids, dates, trials
)

protocol_dicts = pd_convert_to_dict(protocol_blobs)
pd_prepare_dict_for_df(protocol_dicts)

#### SINGLE SESSION ####
session_key = {'sessid': 884257}

prb = (bdata.Sessions & session_key).fetch(
    "protocol_data", as_dict=True
)

s_id, d, t = (bdata.Sessions & session_key).fetch(
    "sessid", "sessiondate", "n_done_trials"
)
prd = pd_convert_to_dict(prb)
pd_prepare_dict_for_df(prd)
session_key = {'sessid': s_id[0]}
pehb = (bdata.ParsedEvents & session_key).fetch("peh", as_dict=True)
pehd = bt.transform_blob(pehb[0]['peh'])



## Next steps
1. Working with single session peh & pd
* ~~load in peh given session_id~~
* ~~convert peh_blob to dict~~
* ensure the lengths of protocol data and peh are the same
* Iterate over trials to grab the peh information & append into new dict
  * maybe validate the sizes again?
* merge new peh and pd dicts together
* convert to df (not cleaned yet)
2. Extend the steps above to be for multiple sessions

3. Write functions that merge peh & pd appropriately 
* test with Training_Progress read in
4. Finally, once working smoothly, clean the df for peh vars

5. Add in additional peh vars

